## 댓글 자연어처리 및 감성분석

#### 1. 필요한 라이브러리 및 파일 불러오기

In [32]:
import pandas as pd
import numpy as np
df = pd.DataFrame(pd.read_csv(r'C:\Users\ASIA_09\Desktop\유통령\retail_ramyun\youtube_comments.csv'))

In [33]:
df= df[['Comments']] 
df

,Comments
0,솔직히먹어봤는데트러플을접해보지못한일반인인나는차이를느끼지못했다...😶
1,이거화사때문에출시된거네ㅋㅋㅋ
2,그냥짜파게티에달걀후라이가젤맛있음
3,김발어디갔는데!!!!!!!!!!!!!!!컵라면엔김발인데!!!!!!!!!!!!!!김발...
4,5:43~5:48오늘자하이라이트입니다ㅋㅋㅋㅋㅋ
...,...
2411,Heisverycuet
2412,Jaeyolnottobemeanoranythingbutthisisn'tveryexo...
2413,X
2414,Fake


#### 2. 한글 제외한 문자 제거

In [34]:
import re 
# '한글'을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")
def preprocess(text):
    text = re.sub(remove_except_ko,' ',text).strip()
    return text

df['Comments'] = df['Comments'].map(lambda x : preprocess(x))

In [35]:
df.head(10)

,Comments
0,솔직히먹어봤는데트러플을접해보지못한일반인인나는차이를느끼지못했다
1,이거화사때문에출시된거네ㅋㅋㅋ
2,그냥짜파게티에달걀후라이가젤맛있음
3,김발어디갔는데 컵라면엔김발인데 김발...
4,오늘자하이라이트입니다ㅋㅋㅋㅋㅋ
5,저는짜파게티맛이예전맛이안나는거같아서항상아쉬웠었는데트러플소스들어가고나서옜날어릴적먹던짜...
6,일반짜파게티컵라면이랑트러플짜파게티컵라면이랑제조방법달라요 이거완전중요뽀인트인디
7,오 컵라면뚜껑에젓가락끼워놓을수있는곳이있는줄몰랐네요ㅋㅋㅋ완전꿀팁이네요근데트러플오일 ...
8,트러플 개모아서한개에넣어서먹오봐요
9,젓가락구멍있는거옛날부터원래있었나요 왜저는 년간몰랐죠


#### 3. OKT 토큰화와 필요없는 조사 제거 

In [41]:
from konlpy.tag import Okt
okt=Okt()
# mecab 사용해 토큰화, 한글자 제거, stop_word 제거
stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']
stopPos = ['Determiner','Adverb','Conjuctions','Josa','PreEomi','Eomi','Suffix',
          'Punctuation','Foreign','Alpha','Number','Unknown','KoreanParticle']
def token_okt(text):
    text = okt.pos(text)
    text = [i for i in text if len(i[0])>1]
    text = [i for i in text if i[0] not in stop_word]
    text = [i[0] for i in text if i[1] not in stopPos]
    return text

In [42]:
# 토큰화 + 토큰리스트 생성 
def make_tokens(df):
    df['tokens'] = ''
    tokens_list=[]
    for i, row in df.iterrows():
        if i%500==0:
            print(i,'/',len(df))
        token = token_okt(df['Comments'][i])
        df['tokens'][i] = ' '.join(token)
    return df

df = make_tokens(df)

0 / 2416
500 / 2416
1000 / 2416
1500 / 2416
2000 / 2416


In [50]:
# 빈칸 찾아내기
df.replace('', np.nan, inplace=True)
df.isnull().sum()

Comments     820
tokens      1443
dtype: int64

In [52]:
#Null 값 없애기
df=df.dropna()
df.isnull().sum()

Comments    0
tokens      0
dtype: int64

In [54]:
df.head(10)

,Comments,tokens
0,솔직히먹어봤는데트러플을접해보지못한일반인인나는차이를느끼지못했다,솔직히 먹어 봤는데 트러플 해보지못 일반인 차이 느끼지못 했다
1,이거화사때문에출시된거네ㅋㅋㅋ,화사 때문 출시 된거
2,그냥짜파게티에달걀후라이가젤맛있음,그냥 짜파게티 달걀후라이 가젤 맛있음
3,김발어디갔는데 컵라면엔김발인데 김발...,김발 어디 갔는데 컵라면 김발 김발 어디 갔냐
4,오늘자하이라이트입니다ㅋㅋㅋㅋㅋ,오늘 하이라이트 입니다
5,저는짜파게티맛이예전맛이안나는거같아서항상아쉬웠었는데트러플소스들어가고나서옜날어릴적먹던짜...,짜파게티 안나 같아서 항상 아쉬웠었는데 트러플 소스 들어가고나서 옜날 어릴 먹던 짜...
6,일반짜파게티컵라면이랑트러플짜파게티컵라면이랑제조방법달라요 이거완전중요뽀인트인디,짜파게티 컵라면 트러플 짜파게티 컵라면 제조 방법 달라 완전 트인디
7,오 컵라면뚜껑에젓가락끼워놓을수있는곳이있는줄몰랐네요ㅋㅋㅋ완전꿀팁이네요근데트러플오일 ...,컵라면 뚜껑 젓가락 끼워놓을수있는 있는줄 몰랐네요 완전 꿀팁 트러플 오일 제조 송로...
8,트러플 개모아서한개에넣어서먹오봐요,트러플 모아 서한 넣어서 먹오 봐요
9,젓가락구멍있는거옛날부터원래있었나요 왜저는 년간몰랐죠,젓가락 구멍 있는거 옛날 원래 있었나요 왜저 몰랐죠


In [56]:
df.to_csv(r'C:\Users\ASIA_09\Desktop\유통령\retail_ramyun\youtube_tokenization.csv')

In [76]:
data = pd.read_csv(r'C:\Users\ASIA_09\Desktop\유통령\retail_ramyun\youtube_tokenization.csv', encoding='cp949')

In [77]:
df_new = pd.DataFrame(data)

In [79]:
df_new = df_new[['tokens']]

In [80]:
df_new

,tokens
0,솔직히 먹어 봤는데 트러플 접해 보지 못 일반인 차이 느끼지 못하다
1,화사 때문 출시 된거
2,그냥 짜파게티 달걀후라이 가 제일 맛있음
3,김발 어디 갔는데 컵라면 김발 김발 어디 갔냐
4,오늘 하이라이트 입니다
...,...
968,먹을만큼만 항상 음식 남기고 먹여주는거하면 서다 버리네 아까운줄 알아야지
969,먹고싶다
970,먹기 싫으면 우리 주더 당신 버리는건 아니지 아까워 먹고싶다
971,친척


#### 4. 감정 사전 불러오기

In [81]:
pos_dict = pd.DataFrame(pd.read_csv(r'C:\Users\ASIA_09\Desktop\KnuSentiLex\pos_pol_word.txt', sep='\n', quoting=3))
pos_dict.head()

,word
0,(-;
1,(^^)
2,(^-^)
3,(^^*
4,(^_^)


In [82]:
neg_dict = pd.DataFrame(pd.read_csv(r'C:\Users\ASIA_09\Desktop\KnuSentiLex\neg_pol_word.txt', sep='\n', quoting=3))
neg_dict.head()

,word
0,가난
1,가난뱅이
2,가난살이
3,가난살이하다
4,가난설음


#### 5. 댓글의 Tone 측정하기

In [102]:
tone_ls = []
for doc in df_new['tokens']:
    #print(doc.split())
    #words=[]
    #words.append(doc.split(' '))
    splitdoc = doc.split()
    pos=0
    neg=0
    for word in splitdoc:
        if word in list(pos_dict['word']):
            pos += 1
        if word in list(neg_dict['word']):
            neg += 1
    if (pos+neg)!=0:
        tone = (pos-neg)/(pos+neg)
        tone_ls.append(tone)

    else:
        tone_ls.append("NaN")

df_new['tone']= tone_ls

C:\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [103]:
df_new.head(10)

,tokens,tone
0,솔직히 먹어 봤는데 트러플 접해 보지 못 일반인 차이 느끼지 못하다,-1
1,화사 때문 출시 된거,NaN
2,그냥 짜파게티 달걀후라이 가 제일 맛있음,NaN
3,김발 어디 갔는데 컵라면 김발 김발 어디 갔냐,NaN
4,오늘 하이라이트 입니다,NaN
5,짜파게티 맛 안나 같아서 항상 아쉬웠었는데 트러플 소스 들어가고 나서 옜날 어릴 먹...,NaN
6,짜파게티 컵라면 트러플 짜파게티 컵라면 제조 방법 달라 완전 포인트인디,NaN
7,컵라면 뚜껑 젓가락 끼워놓을수있는 있는줄 몰랐네요 완전 꿀팁 트러플 오일 제조 송로...,NaN
8,트러플 모아서 한 넣어서 먹어봐요,NaN
9,젓가락 구멍 있는거 옛날 원래 있었나요 왜 저 몰랐다,NaN


tokens    0
tone      0
dtype: int64

(973, 2)